# Transport across sections

`mom6_tools.section_transports` collection of functions for computing and plotting time-series of transports across pre-defined vertical sections.

The goal of this notebook is the following:

1) server as an example on how to post-process the CESM/MOM6 vertical sections defined in diag_table. The location of the current vertical sections computed online can be found at the end of [this notebook](https://gist.github.com/gustavo-marques/bd6e9bb43062dcbc2abda91b757c7c4b); 

2) evaluate model experiments by comparing transports against observed estimates;

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from mom6_tools.section_transports import Transport, options
import matplotlib.pyplot as plt
from mom6_tools.DiagsCase import DiagsCase
import xarray as xr
import yaml, os, numpy

In [3]:
# Read in the yaml file
diag_config_yml_path = "diag_config.yml"
diag_config_yml = yaml.load(open(diag_config_yml_path,'r'), Loader=yaml.Loader)

In [4]:
# Create the case instance
dcase = DiagsCase(diag_config_yml['Case'])
DOUT_S = dcase.get_value('DOUT_S')
if DOUT_S:
  OUTDIR = dcase.get_value('DOUT_S_ROOT')+'/ocn/hist/'
else:
  OUTDIR = dcase.get_value('RUNDIR')+'/'

print('Output directory is:', OUTDIR)
print('Casename is:', dcase.casename)

Output directory is: /glade/scratch/gmarques/archive/g.e23_b15.GJRAv4.TL319_t232_zstar_N65.baseline.001/ocn/hist/
Casename is: g.e23_b15.GJRAv4.TL319_t232_zstar_N65.baseline.001


In [5]:
# load sections where transports are computed online
sections = diag_config_yml['Transports']['sections']

In [6]:
# create an empty class object
class args:
  pass

Define the arguments expected by class "Transport". These have been hard-coded here fow now...

In [7]:
args.case_name = dcase.casename
args.infile = OUTDIR
# set avg dates
avg = diag_config_yml['Avg']
args.start_date = '0001-01-01' # override start date
args.end_date = avg['end_date']
args.label = ''
args.debug = False

Observed flows, more options can be added here

* Griffies et al., 2016: OMIP contribution to CMIP6: experimental and diagnostic protocol for the physical component of the Ocean Model Intercomparison Project. Geosci. Model. Dev., 9, 3231-3296. doi:10.5194/gmd-9-3231-2016

Below we define a function for plotting transport time series. Note the following:

<span style="color:green">green = mean transport is within observed values</span> 

<span style="color:red">red    = mean transport is not within observed values</span> 

<span style="color:gray">gray  = either there isn't observed values to compare with or just a mean value is available (not a range)</span> 

In [8]:
def plotPanel(section,observedFlows=None,colorCode=True):
    ax = plt.subplot(1,1,1)
    color = '#c3c3c3'; obsLabel = None
    if section.label in observedFlows.keys():
      if isinstance(observedFlows[section.label][1:],list) and isinstance(observedFlows[section.label][1:][0],float):
        if colorCode == True:
          if min(observedFlows[section.label][1:]) <= section.data.mean() <= max(observedFlows[section.label][1:]):
            color = '#90ee90'
          else: color = '#f26161'; print('here3')
        obsLabel = str(min(observedFlows[section.label][1:])) + ' to ' + str(max(observedFlows[section.label][1:]))
      else: obsLabel = str(observedFlows[section.label][1:]); print('here4')
    plt.plot(section.time,section.data,color=color)
    plt.title(section.label,fontsize=14)
    plt.text(0.04,0.11,'Mean = '+'{0:.2f}'.format(section.data.mean()),transform=ax.transAxes,fontsize=14)
    if obsLabel is not None: plt.text(0.04,0.04,'Obs. = '+obsLabel,transform=ax.transAxes,fontsize=14)
    if section.ylim is not None: plt.ylim(section.ylim)
    plt.ylabel('Transport (Sv)',fontsize=14); plt.xlabel('Time since beginning of run (yr)',fontsize=14)
    plt.grid()
    return

Plot section transports in alphabetical order

### Agulhas Section

In [10]:
%matplotlib inline
ds = []
agulhas = Transport(args,sections,'h.Agulhas_Section',debug=False); ds.append(agulhas)
plotPanel(agulhas, observedFlows=sections)

Processing  {'h.Agulhas_Section': ['umo', 129.8, 143.6], 'h.Barents_Opening': ['vmo', 2.0], 'h.Bering_Strait': ['vmo', 0.7, 1.1], 'h.Davis_Strait': ['vmo', -2.1, -1.1], 'h.Denmark_Strait': ['vmo', -4.8, -2.0], 'h.Drake_Passage': ['umo', 129.8, 143.6], 'h.English_Channel': ['umo', 0.01, 0.1], 'h.Fram_Strait': ['vmo', -4.7, 0.7], 'h.Florida_Bahamas': ['vmo', 28.9, 34.3], 'h.Florida_Bahamas_extended': ['vmo', 28.9, 34.3], 'h.Florida_Cuba': ['umo', 'None'], 'h.Gibraltar_Strait': ['umo', -1.0, 1.0], 'h.Iceland_Norway': ['vmo', 28.9, 34.3], 'h.Indonesian_Throughflow': ['vmo', -15.0, -13.0], 'h.Mozambique_Channel': ['vmo', -25.6, -7.8], 'h.Pacific_undercurrent': ['umo', 24.5, 28.3], 'h.Taiwan_Luzon': ['umo', -3.0, -1.8], 'h.Windward_Passage': ['vmo', -15.0, 5.0], 'h.Robeson_Channel': ['umo', 'None'], 'h.Yucatan_Channel': ['vmo', 'None'], 'h.Bosporus_Strait': ['vmo', 'None']}


TypeError: can only concatenate str (not "dict") to str

### Bering Strait

In [ ]:
bering = Transport(args, sections, 'h.Bering_Strait'); ds.append(bering)
plotPanel(bering, observedFlows=sections)

### Barents opening

In [ ]:
barents = Transport(args, sections, 'h.Barents_Opening'); ds.append(barents)
plotPanel(barents, observedFlows=sections)

### Davis Strait

In [ ]:
davis = Transport(args, sections,'h.Davis_Strait'); ds.append(davis)
plotPanel(davis, observedFlows=sections)

### Denmark Strait

In [ ]:
denmark = Transport(args, sections,'h.Denmark_Strait'); ds.append(denmark)
plotPanel(denmark, observedFlows=sections)

### Drake Passage

In [ ]:
drake = Transport(args, sections,'h.Drake_Passage'); ds.append(drake)
plotPanel(drake, observedFlows=sections)

### English Channel

In [ ]:
english = Transport(args, sections, 'h.English_Channel'); ds.append(english)
plotPanel(english, observedFlows=sections)

### Fram Strait

In [ ]:
fram = Transport(args, sections, 'h.Fram_Strait'); ds.append(fram)
plotPanel(fram, observedFlows=sections)

### Florida Bahamas

In [ ]:
#florida1 = Transport(args, sections, 'h.Florida_Bahamas', debug=True); ds.append(florida1)
#plotPanel(florida1, observedFlows=sections)

### Florida Bahamas extended

In [ ]:
florida2 = Transport(args, sections, 'h.Florida_Bahamas_extended'); ds.append(florida2)
plotPanel(florida2, observedFlows=sections)

### Florida Cuba

In [ ]:
florida3 = Transport(args, sections, 'h.Florida_Cuba'); ds.append(florida3)
plotPanel(florida3, observedFlows=sections)

### Gibraltar Strait

In [ ]:
gibraltar = Transport(args, sections, 'h.Gibraltar_Strait'); ds.append(gibraltar)
plotPanel(gibraltar, observedFlows=sections)

### Iceland Norway

In [ ]:
iceland = Transport(args, sections, 'h.Iceland_Norway'); ds.append(iceland)
plotPanel(iceland, observedFlows=sections) 

### Indonesian Throughflow

In [ ]:
indo = Transport(args, sections, 'h.Indonesian_Throughflow'); ds.append(indo)
plotPanel(indo, observedFlows=sections)

### Mozambique Channel

In [ ]:
mozambique = Transport(args, sections, 'h.Mozambique_Channel'); ds.append(mozambique)
plotPanel(mozambique, observedFlows=sections)

### Pacific undercurrent

In [ ]:
euc = Transport(args, sections, 'h.Pacific_undercurrent'); ds.append(euc)
plotPanel(euc, observedFlows=sections)

### Taiwan Luzon

In [ ]:
taiwan = Transport(args, sections, 'h.Taiwan_Luzon'); ds.append(taiwan)
plotPanel(taiwan, observedFlows=sections)

### Windward Passage

In [ ]:
windward = Transport(args, sections, 'h.Windward_Passage'); ds.append(windward)
plotPanel(windward, observedFlows=sections)

### Robeson_Channel

In [ ]:
roberson = Transport(args, sections, 'h.Robeson_Channel'); ds.append(roberson)
plotPanel(roberson, observedFlows=sections)

### Yucatan_Channel

In [ ]:
yucatan = Transport(args, sections, 'h.Yucatan_Channel'); ds.append(yucatan)
plotPanel(yucatan, observedFlows=sections)

### Bosporus_Strait

In [ ]:
bosporus = Transport(args, sections, 'h.Bosporus_Strait'); ds.append(bosporus)
plotPanel(bosporus, observedFlows=sections)

### Save netcdf

In [ ]:
print('Saving netCDF file with transports...\n')
if not os.path.isdir('ncfiles'):
      os.system('mkdir -p ncfiles')
        
# create a dataaray
labels = [];
for n in range(len(ds)): labels.append(ds[n].label)
var = numpy.zeros((len(ds),len(ds[0].time)))
ds_out = xr.Dataset(data_vars={ 'transport' : (('sections', 'time'), var)},
                       coords={'sections': labels,
                               'time': ds[0].time})
for n in range(len(ds)):
  ds_out.transport.values[n,:] = ds[n].data

ds_out.to_netcdf('ncfiles/'+dcase.casename+'_section_transports.nc')